<a href="https://colab.research.google.com/github/valyaler/innopolis/blob/master/%D0%94%D0%97_13_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.Взять данные iris.csv

### 2.Загрузить в pyspark

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 55.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=27a955159e91076129e1b519653ef86b91aed89d8e35fe54f5654a06719f1e80
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
# загружаем библиотеки
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark_connection
spark = SparkSession(sc) # open spark session

In [ ]:
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True).csv('iris.csv')
df.show(5)

+------------+-----------+------------+-----------+-------+
|sepal.length|sepal.width|petal.length|petal.width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|         .2| Setosa|
|         4.9|          3|         1.4|         .2| Setosa|
|         4.7|        3.2|         1.3|         .2| Setosa|
|         4.6|        3.1|         1.5|         .2| Setosa|
|           5|        3.6|         1.4|         .2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- sepal.length: string (nullable = true)
 |-- sepal.width: string (nullable = true)
 |-- petal.length: string (nullable = true)
 |-- petal.width: string (nullable = true)
 |-- variety: string (nullable = true)



In [ ]:
#преобразуем названия колонок, создав новый датафрейм
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)

#преобразуем в другие типы данных
df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [ ]:
df_new.show(5) #посмотрим на преобразование

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
df_new.printSchema() #посмотрим на типы атрибутов

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [ ]:
df_new.summary().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|     null|
|    50%|               5.8|                3.0|               4.3|               1.3|     null|
|    75%|               6.4|                3.3|               5.1|               1.8|     null|
|    max|               7.9|  

### 3. При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать PipeLine)

**Векторизация признаков**

Поскольку алгоритмы машинного обучения в PySpark принимают на вход только вектора, то нужно  провести векторизацию. Для преобразования признаков в вектора используется класс VectorAssembler. Объект этого класса принимает в качестве аргументов список с названиями признаков, которые нужно векторизовать (inputCols), и название преобразованного признака (outputCol). После создания объекта VectorAssembler вызывается метод transform.

- Признаки, по которым будет определяться вид: sepal_length, sepal_width, petal_length, petal_width

- Тк признак variety является строковым, а алгоритмы Machine Learning в PySpark работают с числовым значениями, то преобразуем значения столбца variety в числовой вид с помощью StringIndexer: https://spark.apache.org/docs/latest/ml-features#stringindexer

Чтобы указаные шаги делать не один раз, можно их объединить в Pipeline преобразования и применять как единую операцию: https://spark.apache.org/docs/latest/ml-pipeline.html

In [ ]:
pipeline = Pipeline(stages = 
[
  StringIndexer(inputCol='variety', outputCol='varietyInd'),
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')
]
)

In [ ]:
# подгоняем через заданный pipeline наш датасет
pipelineTrained = pipeline.fit(df_new)

In [ ]:
# применяем метод transform для объекта VectorAssembler (преобразуем признаки в вектор) 
pipelineTrained.transform(df_new).show()

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

In [ ]:
# с новым признаком Features создаем новый датафрейм
df_features = pipelineTrained.transform(df_new)

### 4. Разбить данные на train и test

Для решения задач Machine Learning всегда нужно иметь, как минимум, две выборки — обучающую и тестовую. На обучающей мы будем обучать модель, а на тестовой проверять эффективность обученной модели. В PySpark сделать это очень просто, нужно просто вызвать метод randomSplit, который разделит исходный датасет в заданной пропорции. Мы разделим в пропорции 80:20.

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html

In [ ]:
# назначаем соответствующие веса и указываем seed (рандомное разделение ввыборки сохранилось)
train, test = df_features.randomSplit([0.8, 0.2], seed=12345)

In [ ]:
# посмотрим на обучающую выборку
train.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|    Setosa|       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.4|         1.4|        0.3|    Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         4.6|        3.6|         1.0|        0.2|    Setosa|       0.0|[4.6,3.

In [ ]:
test.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|
|         5.1|        3.5|         1.4|        0.3|    Setosa|       0.0|[5.1,3.5,1.4,0.3]|
|         5.4|        3.4|         1.5|        0.4|    Setosa|       0.0|[5.4,3.4,1.5,0.4]|
|         5.4|        3.9|         1.3|        0.4|    Setosa|       0.0|[5.4,3.9,1.3,0.4]|
|         5.7|        2.8|         4.1|        1.3|Versicolor|       1.0|[5.7,2.

### 5. Создать модель логистической регресии и обучить ее

Теперь воспользуемся логистической регрессией (Logistic Regression), которая есть в PySpark, в качестве алгоритма Machine learning. Для этого нужно указать признаки, на которых модель обучается, и признак, который нужно классифицировать. Мы преобразовали признаки цветка в вектор под названием features, поэтому именно его и указываем в аргументе, а целевым признаком будет числовое поле varietyInd

In [ ]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

Осталось только получить предсказания. Для этого вызывается метод transform, который принимает тестовую выборку, а также обучающую (как обучилась модель):

In [ ]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [ ]:
# посмотрим, какие предсказания дает модель на обучающей выборке
train_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|[67.7027919587531...|[1.0,1.6424718354...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|[57.0087434717148...|[1.0,1.4036227809...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|[61.7475610236625...|[1.0,2.4772179810...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|[68.8060989728479...|[1.0,1.2455403381.

In [ ]:
# посмотрим, какие предсказания дает модель на тестовой выборке
test_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|[65.8143591752207...|[1.0,8.9932431201...|       0.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|[52.7723416307807...|[1.0,9.3248066295...|       0.0|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|[64.6690738893863...|[1.0,1.7992914773...|       0.0|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|[71.4022940655650...|[1.0,5.6805112138.

### 6. Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве

Для оценки качества предсказания в spark реализованно несколько классов Если мы решаем задачу бинарной классификации (то есть классов - 2), то нам подойдет BinaryCLassificationEvaluator, а если классов больше 2-х, то MulticlassClassificationEvaluator.

Проверим эффективность модели, используя метрику качества MulticlassClassificationEvaluator, тк признак variety состоит из 3х значений. Нужно лишь указать целевой признак ***varietyInd***, а затем вызвать метод ***evaluate*** и передать в него наши предсказания

In [ ]:
ev = MulticlassClassificationEvaluator(labelCol='varietyInd')

In [ ]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(test_res)*100, '%')

Точность предсказания на обучающей выборке: 100.0 %


In [ ]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%')

Точность предсказания на обучающей выборке: 98.44961240310077 %


 т.к. процент ошибки предсказания на обучающей выборке составил 0% и 1,5% ( что меньше 5%) можем считать качество предсказания надежным